# Examples of usage of solve_ivp

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

ImportError: cannot import name 'solve_ivp'

## 1. Restricted three body problem

The formulation is taken from an introductory chapter to Runge-Kutta methods in "Solving Ordinary Differential Equations I". The system of differential equations describes the 
motion of a body with a neglible mass in a gravitational field of two bodies with masses $\mu$ and $\nu = 1 - \mu$.

$$
y_1'' = y_1 + 2 y_2' - \nu \frac{y_1 + \mu}{D_1} - \mu \frac{y_1 - \mu'}{D_2} \\
y_2'' = y_2 - 2 y_1' - \nu \frac{y_2}{D_1} - \mu \frac{y_2}{D_2} \\
D_1 = ((y_1 + \mu)^2 + y_2^2)^{3/2}, \quad D_2 = ((y_1 - \nu)^2 + y_2^2)^{3/2} \\
\mu = 0.012277471
$$

The initial values are picked such that the solution is periodic with period $x_{end}$:
$$
y_1(0) = 0.994, y_1′(0) = 0, y_2(0) = 0 \\
y_2′(0) = −2.00158510637908252240537862224 \\
x_{end} = 17.0652165601579625588917206249
$$

It is straightforward to rewrite this systems as another first order system. We define the constants and evaluation of the system right-hand side:

In [ ]:
mu = 0.0122771
nu = 1 - mu

a = 0
b = 17.0652165601579625588917206249
ya = [0.994, 0, 0, -2.00158510637908252240537862224]

In [ ]:
def fun(x, y):
    D1 = ((y[0] + mu)**2 + y[1]**2) ** 1.5
    D2 = ((y[0] - nu)**2 + y[1]**2) ** 1.5

    return [
        y[2],
        y[3],
        y[0] + 2 * y[3] - nu * (y[0] + mu) / D1 - mu * (y[0] - nu) / D2,
        y[1] - 2 * y[2] - nu * y[1] / D1 - mu * y[1] / D2
    ]

The ODE is integrated by a single call:

In [ ]:
res = solve_ivp(fun, [a, b], ya, rtol=1e-3, method='RK45')

The function `solve_ivp` provides a continuous solution with the *same accuracy* as underlying method. Here we use is to produce a smooth plot with the compued trajectory.

In [ ]:
xp = np.linspace(a, b, 500)
yp = res.sol(xp)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(yp[0], yp[1], '-')
plt.plot(res.y[0], res.y[1], '.')

We see that at some regions the solver takes quite large steps, but an accurate solution is available at all points nevertheless.

## 2. Bouncing ball

This example illustrates event detection capabilities. The problem is very simple: a ball bounces from the ground and loses some fraction of its vertical velocity.

Define constants.

In [ ]:
g = 9.81  # Gravity.
alpha = 0.8  # Ratio of preserved velocity.
v_hor = 1  # Horizontal velocity.
n_bounces = 10  # Number of bounces.

Define the system right-hand side:

In [ ]:
def fun(x, y):
    return [v_hor, y[2], -g]

Define the bounce event:

In [ ]:
def bounce(x, y):
    return y[1]

bounce.terminate = True  # This event is terminating.
bounce.direction = -1  # Detected when y[1] is decreasing.

The solver runs until the bounce event is detected and then gets restarted with a new value of the vertical velocity.

In [ ]:
ya = [0, 0, 10]  # Start at (0, 0) and with the vertical velocity of 10.

sol_points = []
sol_smooth = []

for i in range(n_bounces):
    # Use a 3-rd order Runge-Kutta method.
    # Integration interval is not relevant. 
    res = solve_ivp(fun, [0, 100], ya, events=bounce, method='RK23')
    sol_points.append(res.y)
    t = np.linspace(res.x[0], res.x[-1], 100)
    sol_smooth.append(res.sol(t))
    ya = res.y[:, -1].copy()
    ya[2] *= -alpha

In [ ]:
sol_points = np.hstack(sol_points)
sol_smooth = np.hstack(sol_smooth)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(sol_smooth[0], sol_smooth[1], '-')
plt.plot(sol_points[0], sol_points[1], 'ro', fillstyle='none')
plt.hlines([0], 0, 10)

## 3. Van der Pol oscillator

The system for Van der Pol oscillator is given as:
$$
y_1' = y_2, \\
y_2' = \mu (1 - y_1^2) y_2 - y_1 \\
y_1(0) = 2, \quad y_2(0) = 0
$$
It becomes stiff for high values of $\mu$, meaning that regions of rapid transition are followed by regions where the solution varies slowly. Explicit methods either diverge or make prohibitevely many steps for stiff problems, thus implicit methods should be used. Our function `solve_ivp` implements a one-step fully implicit Runge-Kutta method of Radau II A family.

We solve this system for $\mu = 10^3$.

In [ ]:
mu = 1e3

def fun(x, y):
    return [y[1], mu * (1 - y[0]**2) * y[1] - y[0]]

In [ ]:
res = solve_ivp(fun, [0, 3000], [2, 0], method='Radau')

In [ ]:
plt.plot(res.x, res.y[0])
plt.plot(res.x, res.y[0], '.')

Note that the solver pass flat regions in very few number of steps. It would not be the case with any explicit method, which will have to make very small steps to overcome the stability issue. For example `method='RK45'` will make more than million steps on this example.